In [55]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [56]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [57]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [58]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [59]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [60]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [61]:
df['loc']

1     (7192 Fisher Island Dr, Miami Beach, FL 33109,...
2     (125 Ocean Dr u0401, Miami Beach, FL 33139, US...
3     (1000 South Pointe Dr, Miami Beach, FL 33139, ...
4     (900 Brickell Key Blvd PH 4, Miami, FL 33131, ...
5     (15701 Collins Ave Suite 4401, Sunny Isles Bea...
6     (19241 Fisher Island Dr, Miami Beach, FL 33109...
7     (100 South Pointe Dr, Miami Beach, FL 33139, U...
8     (300 South Pointe Dr, Miami Beach, FL 33139, U...
9     (1455 Ocean Dr, Miami Beach, FL 33139, USA, (2...
10    (16400 Collins Ave, Sunny Isles Beach, FL 3316...
Name: loc, dtype: object

In [62]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [63]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Palazzo Del Mare\r7192 Fisher Island Dr 7192\r...,2/4/2021,12/20/2021,273,"$15,900,000.00",7025,"$2,263.35",Dora Puig,Elena Bluntzer,Palazzo Del Mare 7192 Fisher Island Dr 7192 Mi...,Palazzo Del Mare,7192 Fisher Island Dr 7192 Miami Beach,"(7192 Fisher Island Dr, Miami Beach, FL 33109,...","(25.762737, -80.13597, 0.0)",25.762737,-80.135970,0.0
2,Ocean House\r125 Ocean Dr U-0401\rMiami Beach,11/5/2021,12/23/2021,40,"$12,000,000.00",4068,"$2,949.85",Pedro Velasco,Christopher Jude,Ocean House 125 Ocean Dr U-0401 Miami Beach,Ocean House,125 Ocean Dr U-0401 Miami Beach,"(125 Ocean Dr u0401, Miami Beach, FL 33139, US...","(25.7700546, -80.1325729, 0.0)",25.770055,-80.132573,0.0
3,Murano at Portofino\r1000 S Pointe Dr 2801\rMi...,11/22/2021,12/21/2021,17,"$8,900,000.00",3365,"$2,644.87",Stacy Robins,Roland Ortiz,Murano at Portofino 1000 S Pointe Dr 2801 Miam...,Murano at Portofino,1000 S Pointe Dr 2801 Miami Beach,"(1000 South Pointe Dr, Miami Beach, FL 33139, ...","(25.7683443, -80.1373909, 0.0)",25.768344,-80.137391,0.0
4,Asia Condominium\r900 Brickell Key Blvd PH4\rM...,4/3/2021,12/24/2021,251,"$7,760,000.00",5359,"$1,448.03",Russell Brooke,Tatiana Zarubina,Asia Condominium 900 Brickell Key Blvd PH4 Miami,Asia Condominium,900 Brickell Key Blvd PH4 Miami,"(900 Brickell Key Blvd PH 4, Miami, FL 33131, ...","(25.769334, -80.1858065, 0.0)",25.769334,-80.185806,0.0
5,Ritz Carlton Sunny Isles\r15701 Collins Ave 41...,8/17/2021,12/20/2021,119,"$6,250,000.00",3204,"$1,950.69",Karen Reiter PA,Diana Garchitorena,Ritz Carlton Sunny Isles 15701 Collins Ave 410...,Ritz Carlton Sunny Isles,15701 Collins Ave 4105 Sunny Isles Beach,"(15701 Collins Ave Suite 4401, Sunny Isles Bea...","(25.9220815, -80.1219976, 0.0)",25.922082,-80.121998,0.0
6,Fisher Island\r19241 Fisher Island Dr 19241\rM...,2/27/2021,12/23/2021,265,"$3,925,000.00",2916,"$1,346.02",Lazaro Rivero,Julian Johnston,Fisher Island 19241 Fisher Island Dr 19241 Mia...,Fisher Island,19241 Fisher Island Dr 19241 Miami Beach,"(19241 Fisher Island Dr, Miami Beach, FL 33109...","(25.7577288, -80.1386049, 0.0)",25.757729,-80.138605,0.0
7,Continuum South\r100 S Pointe Dr 1405\rMiami B...,9/28/2021,12/22/2021,45,"$3,800,000.00",1869,"$2,033.17",Rebecca Feigen,Michael Martirena,Continuum South 100 S Pointe Dr 1405 Miami Beach,Continuum South,100 S Pointe Dr 1405 Miami Beach,"(100 South Pointe Dr, Miami Beach, FL 33139, U...","(25.7679055, -80.1330134, 0.0)",25.767906,-80.133013,0.0
8,Portofino Tower\r300 S Pointe Dr 1403\rMiami B...,10/7/2021,12/20/2021,60,"$2,850,000.00",2340,"$1,217.95",Christian Brandl,Silvia Alvarez,Portofino Tower 300 S Pointe Dr 1403 Miami Beach,Portofino Tower,300 S Pointe Dr 1403 Miami Beach,"(300 South Pointe Dr, Miami Beach, FL 33139, U...","(25.7677055, -80.1348768, 0.0)",25.767706,-80.134877,0.0
9,Il Villagio\r1455 Ocean Dr 704\rMiami Beach,5/26/2021,12/20/2021,187,"$2,150,000.00",1660,"$1,295.18",Ines Flax,Jean-Louis Delbeke,Il Villagio 1455 Ocean Dr 704 Miami Beach,Il Villagio,1455 Ocean Dr 704 Miami Beach,"(1455 Ocean Dr, Miami Beach, FL 33139, USA, (2...","(25.7870529, -80.1291008, 0.0)",25.787053,-80.129101,0.0
10,Oceana IV Condo\r16400 Collins Ave PH46\rSunny...,10/5/2021,12/21/2021,35,"$2,125,000.00",4150,$512.05,Denver Bright,Rohit Anand,Oceana IV Condo 16400 Collins Ave PH46 Sunny I...,Oceana IV Condo,16400 Collins Ave PH46 Sunny Isles Beach,"(16400 Collins Ave, Sunny Isles Beach, FL 3316...","(25.9284667, -80.12379729999999, 0.0)",25.928467,-80.123797,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [64]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [65]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('December 19th - December 25th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['Sq. Ft.'], r['address_only'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}${info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [66]:
m.save('index.html')